In [3]:
import random
import re
import sys
import time
from requests.exceptions import ConnectionError
import bs4
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [22]:
import os 

In [ ]:
base='https://www.sequoiacap.com/companies/'
content={
    "name": [],
    "url": [],
    "description": [],
    "milestones": [],
    "team": [],
    "partner": []
}
delay()
r=requests.get(base)
soup=BeautifulSoup(r.content,'html.parser')

for company in soup.find_all('div',{'class':'companies _company js-company'}):
    
    # parse company name
    name=company.div.text
    
    #send requests to the detail company page 
    #and parse it using Beautifulsoup
    
    try:
        r=requests.get("https://www.sequoiacap.com" + company["data-url"])
        
    except ConnectionError as e:
        print(e)
        r='No response'
    detailed_soup=BeautifulSoup(r.content,'html.parser')
    
    #parse company url
    url=detailed_soup.find_all('a',{'class':'social-link'})
    if len(url)==0:
        url='NA'
    else:
        url=detailed_soup.find('a',{'class':'social-link'})['href']
        
    # Parse company description
    description=detailed_soup.find_all('div',{'class':'company-holder _body-copy -grey-dark'})
    
    if len(description)==0:
        description='NA'
    else:
        description=detailed_soup.find('div',{'class':'company-holder _body-copy -grey-dark'}).p.text
    
    #parse company milestones
    milestones=detailed_soup.find_all(text='Milestones')
    if len(milestones) <=1:
        milestones="NA"
    else:
        milestones=(detailed_soup.find(text='Milestones').parent.parent.ul.text.strip().replace('\n',';'))
    
    #parse company founders
    team=detailed_soup.find_all(text='Team')
    if len(team) <=1:
        team="NA"
    else:
        team=( detailed_soup.find(text='Team').parent.parent.ul.text.strip().replace('\n',';'))
    
    #parse sequoia partner responsible for the company
    partner=detailed_soup.find_all(text=re.compile('^Partners?$'))
    if len(partner):
        partner='NA'
    else:
        partner=(detailed_soup.find(text=re.compile('^Partners?$')).parent.parent.ul.text.strip().replace('\n',';'))
    
    
    #append all data belonging to this company
    #to the content dictionary.
    content['name'].append(name)
    content['url'].append(url)
    content["description"].append(description)
    content["milestones"].append(milestones)
    content["team"].append(team)
    content["partner"].append(partner)
    delay()

In [7]:
df=pd.DataFrame(content)

In [9]:
df=df.replace('',np.nan).fillna(value='NA')

# INDEED

In [92]:
def handle_null(element):
    if type(element) != str:
        return element.text.replace('\n',' ').strip()
    else:
        return element
    
def delay():
    time.sleep(random.uniform(0,2))
    return None

In [20]:
response=requests.get('https://mx.indeed.com/browsejobs?_ga=2.52472939.2127082735.1605584059-1200163399.1577333572')
soup_object=BeautifulSoup(response.content)
categorias=soup_object.find_all('li',{'class':'category'})
categ_list=[categoria.a.text for categoria in categorias]

In [93]:
for categoria in categ_list:
    tit_list=[]
    comp_list=[]
    web_url=f'https://mx.indeed.com/jobs?q={categoria}&limit=50&fromage=14&start='
    for i in range(0,500,50):
        web_url=web_url+str(i)
        response=requests.get(web_url)        
        soup_object=BeautifulSoup(response.content)
        titles=soup_object.find_all('a','jobtitle turnstileLink')
        parents=soup_object.find_all('div',{'class':'sjcl'})
        
        
        companies=[]
        for element in parents:
            if not element.find('span',{'class':'company'}):
                companies.append('NA')
            else:
                companies.append(element.find('span',{'class':'company'}))
                
        tit_list=tit_list+[title.text.replace('\n',' ').strip() for title in titles]
        comp_list=comp_list+[handle_null(comp) for comp in companies]
        delay()
    
    direct='alljobs_categorias/'
    if not os.path.exists(direct):
        os.makedirs(direct)
    jobs=pd.DataFrame({'titulos':tit_list,'companias':comp_list}).drop_duplicates().reset_index(drop=True)
    jobs.to_csv(os.path.join(direct,categoria),index=False)